This experiment shows that a simple neural network can learn to predict the difference between a JPG and PNG image. Therefore, models trained on datasets where one class of images are JPGs and another are PNGs are very likely to be able to predict the difference between the two solely on the image format and not the image content.

Specifically, this experiment should challenge the assumptions of the journal article ["Criminal tendency detection from facial images and the gender bias effect"](https://journalofbigdata.springeropen.com/articles/10.1186/s40537-019-0282-4) which used all PNGs for mugshots of (presumably) criminals and all JPGs for photos of (presumably) non-criminals. The seemingly excellent result of their experiment (97% accuracy) can be discarded now that we understand that what they created was probably a very sophisticated PNG vs JPG detector.

This notebook is based on the example from the [Deep Learning With Python](https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/2.1-a-first-look-at-a-neural-network.ipynb) Chapter 2 notebook on handwritted image detection.

In [1]:
import keras
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

Using TensorFlow backend.


Function for converting the format of an image into either JPG or PNG

In [2]:
from PIL import Image
import numpy as np
from io import BytesIO

def convertFormat(ndarray, imgFormat):
    acc_list = []
    
    imgArray=np.reshape(ndarray,(28,28))
    image = Image.fromarray(imgArray).convert('L')

    with BytesIO() as f:
        image.save(f, format=imgFormat)
        data = np.asarray(Image.open(f), dtype="float32").ravel()
        acc_list.append(data)

    return np.array(acc_list)


Function for training and evaluating a neural network with test and train datasets and test and train labels

In [3]:
from keras import models
from keras import layers
from keras.utils import to_categorical

def trainAndEvalNetwork(train_data, train_labels, test_data, test_labels, input_size, label_range):
    network = models.Sequential()
    network.add(layers.Dense(512, activation='relu', input_shape=(input_size,)))
    network.add(layers.Dense(label_range, activation='softmax'))
    
    network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    
    train_data = train_data.reshape((len(train_data), input_size))
    train_data = train_data.astype('float32') / 255

    test_data = test_data.reshape((len(test_data), input_size))
    test_data = test_data.astype('float32') / 255
    
    network.fit(train_data, to_categorical(train_labels), epochs=5, batch_size=128)
    
    test_loss, test_acc = network.evaluate(test_data, to_categorical(test_labels))
    
    print('test_acc:', test_acc)

Randomly designate each image as either 0 for "Non-criminal" or 1 for "Criminal". These are handwritten digits, but we designate them completely randomly and with no consideration for the actual content of the image.

In [4]:
# Random junk experiment
import random

random.seed()

train_labels_rand = []
for i in range (0, len(train_images)):
    train_labels_rand.append(random.randrange(0,2))
    
test_labels_rand = []
for i in range (0, len(test_images)):
    test_labels_rand.append(random.randrange(0,2))

Train and evaluate the model on the dataset of handwritten numbers, with our new randomly generated labels designating them either Criminal or Non-criminal

In [5]:
trainAndEvalNetwork(train_images, train_labels_rand, test_images, test_labels_rand, 28 * 28, 2)






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/5





60000/60000 [==============================] - 4s 61us/step - loss: 0.7043 - acc: 0.4969
Epoch 2/5
60000/60000 [==============================] - 3s 52us/step - loss: 0.6941 - acc: 0.5118
Epoch 3/5
60000/60000 [==============================] - 3s 53us/step - loss: 0.6922 - acc: 0.5233
Epoch 4/5
60000/60000 [==============================] - 3s 54us/step - loss: 0.6889 - acc: 0.5352
Epoch 5/5
10000/10000 [==============================] - 0s 41us/step
test_acc: 0.5053


Note that the above result is around 50%. Essentially, our model is guessing one of two possible values and is doing no better than a coin toss.

Next, we produce a modified dataset where the images we have labelled as 0 (aka Non-criminal) are converted to JPGs and those labelled as 1 (aka Criminal) are converted to PNGs.

In [6]:
train_images_crim = []
for index, img in enumerate(train_images):
    if train_labels_rand[index] == 0:
        train_images_crim.append(convertFormat(img, 'JPEG'))
    else:
        train_images_crim.append(convertFormat(img, 'PNG'))

test_images_crim = []
for index, img in enumerate(test_images):
    if test_labels_rand[index] == 0:
        test_images_crim.append(convertFormat(img, 'JPEG'))
    else:
        test_images_crim.append(convertFormat(img, 'PNG'))

train_images_crim = np.array(train_images_crim)
test_images_crim = np.array(test_images_crim)

Train and evaluate the model on the above dataset with "criminal" and "non-criminal" images, labelled accordingly.

In [7]:
trainAndEvalNetwork(train_images_crim, train_labels_rand, test_images_crim, test_labels_rand, 28 * 28, 2)

Epoch 1/5
60000/60000 [==============================] - 4s 62us/step - loss: 0.3849 - acc: 0.8159
Epoch 2/5
60000/60000 [==============================] - 3s 51us/step - loss: 0.0897 - acc: 0.9732
Epoch 3/5
60000/60000 [==============================] - 3s 51us/step - loss: 0.0333 - acc: 0.9904
Epoch 4/5
60000/60000 [==============================] - 3s 51us/step - loss: 0.0155 - acc: 0.9957
Epoch 5/5
10000/10000 [==============================] - 0s 45us/step
test_acc: 0.9946


Note the above accuracy value - approximately 99% - shows that we have trained a model to determine whether the image is a JPG or PNG. The fact that we have labelled one as criminal and the other as non-criminal is irrelevant and is merely confusing our understanding of what the model is actually doing. A neural network that trains off of two datasets, where one happens to be all PNGs and the other happens to be all JPGs, will have an artificially high accuracy rate.